In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

In [5]:
books = sqlContext.read.csv('books.csv',header=True)

In [6]:
books.show()

+---+--------+------------+--------+-----------+----------+--------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id| book_id|best_book_id| work_id|books_count|      isbn|  isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+--------+------------+--------+-----------+----------+--------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------

In [7]:
books.registerTempTable("df")

In [8]:
#Serach book by book id 
qry1 = spark.sql("select * from df where book_id = 3")

In [9]:
qry1.show()

+---+-------+------------+-------+-----------+---------+--------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id|book_id|best_book_id|work_id|books_count|     isbn|  isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+-------+------------+-------+-----------+---------+--------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
|

In [10]:
#Query by top 10 rating/ Display average rating of each book
#To dispaly all books drop the limit
qry2 = spark.sql("select authors,original_publication_year,title,language_code,average_rating from df order by average_rating DESC limit 10")

In [11]:
qry2.show()

+--------------------+-------------------------+--------------------+-------------+--------------+
|             authors|original_publication_year|               title|language_code|average_rating|
+--------------------+-------------------------+--------------------+-------------+--------------+
|      Bill Watterson|                     2005|The Complete Calv...|          eng|          4.82|
|   Brandon Sanderson|                     2014|Words of Radiance...|          eng|          4.77|
|J.K. Rowling  Mar...|                     2003|Harry Potter Boxe...|          eng|          4.77|
|Anonymous  Lane T...|                     2002|     ESV Study Bible|          eng|          4.76|
|     Francine Rivers|                     1993|Mark of the Lion ...|        en-US|          4.76|
|      Bill Watterson|                     1996|It's a Magical Wo...|          eng|          4.75|
|        J.K. Rowling|                     1998|Harry Potter Boxs...|          eng|          4.74|
|      Bil

In [12]:
#Query by bottom 10 rating
qry3 = spark.sql("select authors,original_publication_year,title,language_code,average_rating from df order by average_rating limit 10")

In [13]:
qry3.show()

+--------------------+-------------------------+--------------------+-------------+--------------+
|             authors|original_publication_year|               title|language_code|average_rating|
+--------------------+-------------------------+--------------------+-------------+--------------+
|       Chetan Bhagat|                     2005|One Night at the ...|          eng|          2.47|
|        Alice Sebold|                     2007|     The Almost Moon|          eng|          2.67|
|     Howard Jacobson|                     2010|The Finkler Question|          eng|          2.76|
|    Candace Bushnell|                     2000|        Four Blondes|         null|           2.8|
|Gregory Maguire  ...|                     2001|                Lost|          eng|           2.8|
|      Julia Pierpont|                     2015|Among the Ten Tho...|          eng|          2.84|
|     Chuck Palahniuk|                     2010|            Tell-All|         null|          2.84|
|   Lauren

In [14]:
#Find a book by authors name 
qry4 = spark.sql("select title,original_publication_year,language_code,authors from df where authors = 'Suzanne Collins' order by original_publication_year + 0 DESC ")

In [15]:
qry4.show()

+--------------------+-------------------------+-------------+---------------+
|               title|original_publication_year|language_code|        authors|
+--------------------+-------------------------+-------------+---------------+
|Mockingjay (The H...|                     2010|          eng|Suzanne Collins|
|The Hunger Games ...|                     2010|          eng|Suzanne Collins|
|Catching Fire (Th...|                     2009|          eng|Suzanne Collins|
|The Hunger Games ...|                     2008|          eng|Suzanne Collins|
|Gregor and the Co...|                     2007|         null|Suzanne Collins|
|Gregor and the Ma...|                     2006|          eng|Suzanne Collins|
|Gregor and the Cu...|                     2005|          eng|Suzanne Collins|
|Gregor and the Pr...|                     2004|          eng|Suzanne Collins|
|Gregor the Overla...|                     2003|          eng|Suzanne Collins|
+--------------------+-------------------------+----

In [16]:
#Top books by popularity calcuated using maximum ratings_count
#ratings_count gives the idea about books that has been rated by maximum number of people 
#Popular book recommendation
qry5 = spark.sql("select title,original_publication_year,language_code,authors,average_rating from df order by ratings_count limit 10")

In [17]:
qry5.show()

+--------------------+-------------------------+-------------+-------------------+--------------+
|               title|original_publication_year|language_code|            authors|average_rating|
+--------------------+-------------------------+-------------+-------------------+--------------+
|Carter Beats the ...|                     2001|          eng|    Glen David Gold|          4.09|
|           Leviathan|                     1992|          eng|        Paul Auster|          3.95|
|        Coming Clean|                     2013|        en-US|Kimberly Rae Miller|          3.88|
|     Keys to Drawing|                     1985|          eng|        Bert Dodson|             4|
|       Dear Daughter|                     2014|          eng|   Elizabeth Little|          3.44|
|Hope to Die (Alex...|                     2014|          eng|    James Patterson|          4.14|
|           Cape Fear|                     1958|         null|  John D. MacDonald|          4.07|
|         Most Wante

In [18]:
#Books with similar rating
qry6 = spark.sql("select title,original_publication_year,language_code,authors,average_rating from df where average_rating between 4 and 5 order by original_publication_year + 0 DESC ")

In [19]:
qry6.show()

+--------------------+-------------------------+-------------+-----------------+--------------+
|               title|original_publication_year|language_code|          authors|average_rating|
+--------------------+-------------------------+-------------+-----------------+--------------+
|     The Hate U Give|                     2017|          eng|     Angie Thomas|          4.62|
|A Court of Wings ...|                     2017|          eng|    Sarah J. Maas|          4.54|
|           Egomaniac|                     2017|         null|       Vi Keeland|          4.34|
|King's Cage (Red ...|                     2017|          eng| Victoria Aveyard|          4.03|
|     Norse Mythology|                     2017|          eng|      Neil Gaiman|          4.12|
|The Dry (Aaron Fa...|                     2017|          eng|      Jane Harper|          4.07|
|            Faithful|                     2017|          eng|    Alice Hoffman|          4.02|
|End of Watch (Bil...|                  

In [20]:
#Popular book by reviews
# +0 forces a implicit conversion refer- StackOverFlow
qry7 = spark.sql("select title,original_publication_year,language_code,authors,work_text_reviews_count from df order by work_text_reviews_count + 0 DESC limit 10")

In [21]:
qry7.show()

+--------------------+-------------------------+-------------+----------------+-----------------------+
|               title|original_publication_year|language_code|         authors|work_text_reviews_count|
+--------------------+-------------------------+-------------+----------------+-----------------------+
|The Hunger Games ...|                     2008|          eng| Suzanne Collins|                 155254|
|The Fault in Our ...|                     2012|          eng|      John Green|                 140739|
|           Gone Girl|                     2012|          eng|   Gillian Flynn|                 121614|
|Divergent (Diverg...|                     2011|          eng|   Veronica Roth|                 101023|
|Mockingjay (The H...|                     2010|          eng| Suzanne Collins|                  96274|
|Twilight (Twiligh...|                     2005|        en-US| Stephenie Meyer|                  95009|
|      The Book Thief|                     2005|          eng|  